In [1]:
import pandas as pd
import numpy as np
import random
import pickle
import numba

In [2]:
cm = pd.read_csv('final.csv')
cm['genres'] = cm.genres.str.split('/')
cm['tags'] = cm.tags.str.split('\|\|')
mymv = set(cm.movieId)

In [4]:
cm.iloc[:,0]

0        180065
1        180067
2        180071
3        180075
4        180077
          ...  
25108    209073
25109    209085
25110    209123
25111    209129
25112    209169
Name: movieId, Length: 25113, dtype: int64

In [ ]:
s = set()
for it in cm['rating']:
    s |= set([it])

In [6]:
with open('movie-id-map.pkl', 'rb') as f:
    mv2id = pickle.load(f)
with open('movie-id-map2.pkl', 'rb') as f:
    mv2id2 = pickle.load(f)

In [23]:
mymv = list(mymv)
mv2id = {mymv[i]: i for i in range(len(mymv))}
with open('movie-id-map.pkl', 'wb') as of:
    pickle.dump(mv2id, of)

In [7]:
def calculate_similarity(cname: str):
    dist = set()
    for it in cm[cname]:
        dist |= set(it)
    print(f"len {len(dist)}")
    dist = list(dist)
    p2id = {dist[i]: i for i in range(len(dist))}
    psim = np.zeros((len(dist), len(mymv)), dtype='<f2')
    for mv, it in cm[['movieId', cname]].itertuples(index=False):
        for i in it:
            psim[p2id[i], mv2id[mv]] += 1
    print('done psim')
    return psim
    msim = np.zeros((len(mymv), len(mymv)), dtype='<f2')
    for i in range(len(mymv)):
        msim[:, i] += np.linalg.norm(psim[:, np.newaxis, i]-psim, axis=0)
        if i%1000 == 0:
            print(f'done{i}')
    return msim

In [ ]:
np.savez_compressed('director-similarity.npz', calculate_similarity('director'))

In [ ]:
def big_calculate_similarity(cname: str):
    dist = set()
    for it in cm[cname]:
        dist |= set(it)
    print(f"len {len(dist)}")
    dist = list(dist)
    p2id = {dist[i]: i for i in range(len(dist))}
    f = h5py.File('temp.hdf5', 'w')
    psim = f.create_dataset(cname+'-similarity', (len(dist), len(mymv)), dtype='f2')
    i = 0
    for mv, it in cm[['movieId', cname]].itertuples(index=False):
        psim[np.sort(np.array([p2id[i] for i in it])), mv2id[mv]] = 1
        i += 1
        if i%1000 == 0:
            print(f'psim {i}')
    print('done psim')
    f.close()
    return
    final = h5py.File('similarities.hdf5', 'w')
    msim = final.create_dataset(cname+'-similarity', (len(mymv), len(mymv)), dtype='f2')
    for i in range(len(mymv)):
        msim[:, i] += np.linalg.norm(psim[:, i][:, np.newaxis]-psim, axis=0)
        if i%1000 == 0:
            print(f'done{i}')
    f.close()
    final.close()

In [ ]:
np.savez_compressed('genre-similarity.npz', calculate_similarity('genres'))

In [9]:
def rating_similarity():
    rsim = np.zeros((len(mymv), len(mymv)), dtype='<f2')
    i = 0
    for mv, rt in cm[['movieId', 'rating']].itertuples(index=False):
        rsim[mv2id[mv], :] += rt
        rsim[:, mv2id[mv]] -= rt
        i += 1
        if i%1000 == 0:
            print('done', i)
    return rsim

In [ ]:
np.savez_compressed('rating-similarity.npz', rating_similarity())

In [ ]:
final['cast-similarity']

In [10]:
def half_similarity(cname: str):
    print('s')
    rf = h5py.File('/media/aayush/Acer/UbuntuStuff/temp.hdf5', 'r')
    print('l1')
    psim = rf[cname+'-similarity']
    print('l2')
    final = h5py.File('/media/aayush/Acer/UbuntuStuff/similarities.hdf5', 'w')
    print('l3')
    msim = final.create_dataset(cname+'-similarity', (len(mymv), len(mymv)), dtype='f2')
    print('l4')
    for i in range(len(mymv)-1):
        print('start', i)
        row = rf[cname+'-similarity'][:, i]
        for j in range(i+1, len(mymv)):
            print('mid', j)
            msim[i, j] = np.linalg.norm(row - rf[cname+'-similarity'][:, j])
    rf.close()
    final.close()

In [ ]:
big_calculate_similarity('cast')


In [ ]:
half_similarity('cast')

In [1]:
def other_calculate_similarity(cname: str):
    dist = set()
    for it in cm[cname].dropna():
        dist |= set(it)
    print(f"len {len(dist)}")
    dist = list(dist)
    p2id = {dist[i]: i for i in range(len(dist))}
    psim = np.zeros((len(dist), len(mymv)), dtype='f4')
    for mv, it in cm[['movieId', cname]].dropna().itertuples(index=False):
        psim[[p2id[i] for i in it], mv2id[mv]] += 1
    print('done psim')
    return psim
    msim = np.zeros((len(mymv), len(mymv)), dtype='<f2')
    for i in range(len(mymv)):
        msim[:, i] += np.linalg.norm(psim[:, np.newaxis, i]-psim, axis=0)
        print(f'done{i}')
    return msim

In [12]:
@numba.jit(nopython=True, parallel=True)
def other_half_similarity(sim, nmv, start, end):
    print('Enter')
    msim = np.zeros(shape=(nmv, nmv), dtype=numba.float32)
    for i in range(start, end):
        col = sim[:, i]
        for j in range(i+1, nmv):
            msim[i, j] += np.linalg.norm(col-sim[:, j])
        print('done', i)
    return msim

In [9]:
psim = other_calculate_similarity('tags')

len 24529
done psim


In [ ]:
other_half_similarity(psim, len(mymv), 0, 10)

In [19]:
def subset_similarity(cname: int, movies: list):
    subset = cm[cm.movieId.isin(set(movies))].dropna()
    dist = set()
    for it in subset[cname]:
        dist |= set(it)
    dist = list(dist)
    print('done set')
    p_to_ind = {dist[i]: i for i in range(len(dist))}
    movie_to_ind = {movies[i]: i for i in range(len(movies))}
    psim = np.zeros((len(dist), len(movies)), dtype='<f2')
    for mv, it in subset[['movieId', cname]].itertuples(index=False):
        psim[[p_to_ind[i] for i in it], movie_to_ind[mv]] += 1
    print('done psim')
    msim = np.zeros((len(movies), len(movies)), dtype='<f2')
    for i in range(len(movies)):
        msim[:, i] += np.linalg.norm(psim[:, np.newaxis, i]-psim, axis=0)
#         if i%100 == 0:
        print('msim', i)
    return msim

In [15]:
umv = random.sample(mymv, 3000)

In [ ]:
mat = subset_similarity('tags', umv)

In [3]:
with np.load('all_similarities/tags-similarity.npz', 'r') as npf:
    simm = npf['arr_0']

In [4]:
df = pd.read_csv('Movie_data_cleaned.csv')
nmv = list(df.movieId)

In [5]:
cols = [mv2id[m] for m in nmv]

In [6]:
len(cols)

23843

In [7]:
cols.sort()

In [ ]:
simm = simm[np.vstack(cols), cols]

In [16]:
simm.shape

(23843, 23843)

In [21]:
np.savez_compressed('genre-similarity.npz', simm)

In [40]:
with open('movie-id-map2.pkl', 'wb') as of:
    pickle.dump(mv2id2, of)

In [3]:
with open('Test_Set_users_list.csv', 'r') as f:
    uids = [int(x) for x in f.readlines()]

In [4]:
len(uids)

13758

In [21]:
us = pd.read_csv('ml-25m/timeless_ratings.csv')
us = us[us.userId.isin(uids)]
us = us[us.movieId.isin(mv2id2)]

In [22]:
us.shape

(172292, 3)

In [23]:
uids = np.array(uids)

In [24]:
rtu = np.array(us.userId)

In [25]:
rtm = np.array(us.movieId)

In [26]:
rtr = np.array(us.rating)

In [37]:
@numba.njit()
def makeumat(uu, nmv, start, end):
    msim = np.zeros((nmv, nmv))
    for i in range(start, end):
        col = uu[:, i]
        for j in range(i+1, nmv):
            msim[i, j] += np.linalg.norm(col-uu[:, j])
        if i%1000 == 0:
            print('done', i)
#     del uu
    return msim

In [ ]:
msim = makeumat(uu, len(mv2id2))

In [27]:
uu = np.zeros((len(uids), len(mv2id2)))
u2id = {uids[i]:i for i in range(len(uids))}
for i in range(len(rtu)):
    uu[u2id[rtu[i]], mv2id2[rtm[i]]] = rtr[i]

In [7]:
psim = other_calculate_similarity('tags')

len 24529
done psim


In [10]:
psim[:, mv2id[208711]].sum()

2.0

In [11]:
psim[:, mv2id[180083]].sum()

1.0

In [14]:
np.linalg.norm(psim[:, mv2id[208711]]-psim[:, mv2id[180083]])

1.7320508

In [15]:
(psim[:, mv2id[208711]]-psim[:, mv2id[180083]]).sum()

1.0

In [18]:
x = np.reciprocal((np.arange(9.)+1).reshape((3, 3)))

In [22]:
x -= 0.1

In [23]:
x

array([[0.9       , 0.4       , 0.23333333],
       [0.15      , 0.1       , 0.06666667],
       [0.04285714, 0.025     , 0.01111111]])

In [24]:
np.linalg.norm(x)

1.0314911925813472